## 최종 제출물은 약 1000만(9790K Pairs)개의 코드쌍을 학습시킨 결과입니다.
### 각 모델 별 세부 전략에 사용된 코드는 다음과 같습니다.

## Improved R-Drop Trainer for GraphCodeRoberta with RBERT

### "R-Drop: Regularized Dropout for Neural Networks"(https://arxiv.org/abs/2106.14448)

트랜스포머 내부의 Dropout은 forward passing 때마다 랜덤하게 분포합니다. 이때 어텐션 블록들을 각자 독립적으로 통과한 두 개의 Logit을 바탕으로 KL Divergence Loss를 최소화시키면 성능향상이 있습니다. 저희 팀에서는 이때 논문대로 동일한 코드쌍을 단순히 두 번 통과시키지는 않았습니다. 앞뒤 순서를 바꾼 코드쌍: "Code1[SEP][SEP]Code2"와 "Code2[SEP][SEP]Code1"를 각자 한 번씩 통과시키면서 R-Drop Regularization을 진행했고, 이를 통해 성능을 올렸습니다.

In [ ]:
class RobertaRBERT(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.net = nn.Sequential(
            nn.Dropout(config.hidden_dropout_prob),
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(config.hidden_size * 4, config.num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        hidden_states = outputs[0]
        batch_size, _, hidden_size = hidden_states.shape    

        # CLS code1 SEP SEP code2 SEP
        cls_flag = input_ids == self.config.tokenizer_cls_token_id  # cls token
        sep_flag = input_ids == self.config.tokenizer_sep_token_id  # sep token

        special_token_states = hidden_states[cls_flag + sep_flag].view(batch_size, -1, hidden_size)  # (batch_size, 4, hidden_size)
        special_hidden_states = self.net(special_token_states)  # (batch_size, 4, hidden_size)

        pooled_output = special_hidden_states.view(batch_size, -1)  # (batch_size, hidden_size * 4)
        logits = self.classifier(pooled_output)  # (batch_size, num_labels)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict, Any, List, Optional, Union, Tuple
from transformers import Trainer
from transformers.trainer_pt_utils import nested_detach
from transformers.utils import logging

logger = logging.get_logger(__name__)


class ImprovedRDropTrainer(Trainer):
    def get_kl_loss(self, loss_fn, logits_1, logits_2, alpha=1):
        loss_kl_1 = loss_fn(F.log_softmax(logits_1, dim=-1), F.softmax(logits_2, dim=-1))
        loss_kl_2 = loss_fn(F.log_softmax(logits_2, dim=-1), F.softmax(logits_1, dim=-1))
        return alpha * (loss_kl_1 + loss_kl_2) / 2

    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:

        has_labels = all(inputs.get(k) is not None for k in self.label_names)
        inputs = self._prepare_inputs(inputs)
        if ignore_keys is None:
            if hasattr(self.model, "config"):
                ignore_keys = getattr(self.model.config, "keys_to_ignore_at_inference", [])
            else:
                ignore_keys = []

        # labels may be popped when computing the loss (label smoothing for instance) so we grab them first.
        if has_labels:
            labels = nested_detach(tuple(inputs.get(name) for name in self.label_names))
            if len(labels) == 1:
                labels = labels[0]
        else:
            labels = None

        with torch.no_grad():
            if has_labels:
                with self.autocast_smart_context_manager():
                    loss, logits = self.compute_eval_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()

            else:
                loss = None
                with self.autocast_smart_context_manager():
                    outputs1 = model(
                        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
                    )
                    outputs2 = model(
                        input_ids=inputs["input_ids2"], attention_mask=inputs["attention_mask2"]
                    )

                    logits = (outputs1.logits + outputs2.logits) / 2

        if prediction_loss_only:
            return (loss, None, None)

        if len(logits) == 1:
            logits = logits[0]

        return (loss, logits, labels)

    def compute_loss(self, model, inputs):
        num_labels = 2
        labels = inputs.pop("labels")

        # cls code1 sep sep code2 sep
        outputs1 = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits1 = outputs1.logits

        # cls code2 sep sep code1 sep
        outputs2 = model(input_ids=inputs["input_ids2"], attention_mask=inputs["attention_mask2"])
        logits2 = outputs2.logits

        # Crossentropy Loss
        loss_fct_1 = nn.CrossEntropyLoss()
        loss_nll = (
            loss_fct_1(logits1.view(-1, num_labels), labels.view(-1))
            + loss_fct_1(logits2.view(-1, num_labels), labels.view(-1))
        ) / 2

        # KL-Divergence Loss
        loss_fct_2 = nn.KLDivLoss(reduction="batchmean")
        loss_kl = self.get_kl_loss(loss_fct_2, logits1, logits2)
        return loss_nll + loss_kl

    def compute_eval_loss(self, model, inputs, return_outputs=False):
        num_labels = 2
        labels = inputs.pop("labels")

        outputs1 = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        outputs2 = model(input_ids=inputs["input_ids2"], attention_mask=inputs["attention_mask2"])

        logits = (outputs1.logits + outputs2.logits) / 2

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

        return (loss, logits) if return_outputs else loss


## PLBART-Large Encoder

### "RBERT: Enriching Pre-Training Language Model with Entity Information for Relation Classification" (https://arxiv.org/pdf/1905.08284.pdf)
### "Unified Pre-training for Program Understanding and Generation"(https://arxiv.org/abs/2103.06333.pdf)


GraphCodeRobera의 경우는 모델은 훌륭하게 기학습되었으나 토크나이저가 비효율적입니다. 데이터셋의 75%만이 512 토큰 안에 들어갑니다. 데이터셋이 지나치게 잘게 쪼개진다는 단점을 저희 팀에서는 어떻게든 극복해보려고 했습니다. 또한 Base 모델 밖에 제공이 안 된다는 단점이 있습니다. 

이에 반해 PLBart 같은 경우는 Base 모델이 아니라 Large 모델까지 제공하며,  임베딩이 깊이가 1024 차원으로써 더 많은 내용을 담고 있습니다. 무엇보다 92%의 데이터셋이 512 토큰안에 들어갑니다. 그러나, 12 Layer Encoder - 12 Layer Decoder이라는 점에서 모델이 지나치게 크기 때문에 훈련시키기에는 비효율적이라는 단점이 있습니다. 따라서 Encoder - Decoder 모델을 전부 사용하는 것보다는, Encoder 부분만 따와서 사용했으며 이 방법이 Batch size를 늘리기에도 좋고 훈련도 빠르게 할 수 있었습니다. 

BART는 다른 모델들과는 다르게, CLS 토큰이 없습니다. 보통 BART로 분류작업을 할 때, SEP토큰과 EOS 토큰이 동일하기 때문에 마지막 토큰을 추출하여서 분류를 진행합니다. 이때, PLBART의 논문에서 특이하게 "python" 토큰도 집어넣어서 기학습을 진행했다는 걸 확인했습니다. 또한 92%의 데이터가 전부 512토큰 안에 들어간다는 점을 감안하여 Python Token을 포함한 Code1 문장, Code2 문장 단위로 Token들의 hidden state을 평균시켜서 RBERT 방식으로 분류를 진행했습니다.


In [ ]:
class VHBartEncoderForSequenceClassification(PLBartModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.entity_fc_layer = FCLayer(
            self.config.hidden_size, self.config.hidden_size, self.config.dropout_rate
        )
        self.proj_fc_layer = FCLayer(
            self.config.hidden_size * 4, self.config.hidden_size, self.config.dropout_rate
        )
        self.label_classifier = FCLayer(
            self.config.hidden_size * 3,
            self.config.num_labels,
            self.config.dropout_rate,
            use_activation=True,
        )

    def forward(
        self, input_ids, attention_mask, code1_mask, code2_mask, last_token_index, labels=None,
    ):

        outputs = self.encoder(
            input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,
        )

        # Global token's 4 hidden states concatenation and  projection result
        idx_seq = torch.arange(input_ids.size(0)).to(input_ids.device)
        # print(idx_seq)
        cls_concat = torch.cat(
            tuple(
                [outputs["hidden_states"][i][idx_seq, last_token_index] for i in [-4, -3, -2, -1]]
            ),
            dim=-1,
        )
        cls_output = self.proj_fc_layer(cls_concat)

        # Global average on sentences
        sequence_output = outputs["last_hidden_state"]

        code1_sentence_h = self.entity_average(
            sequence_output, code1_mask
        )  # token in between code1 entities ->
        code1_sentence_h = self.entity_fc_layer(
            code1_sentence_h
        )  # code1 entity's fully connected layer | yellow on diagram

        # Average on code2 sentence
        code2_sentence_h = self.entity_average(
            sequence_output, code2_mask
        )  # token in between code2 entities
        code2_sentence_h = self.entity_fc_layer(
            code2_sentence_h
        )  # code2 entity's fully connected layer | red on diagram

        # Concat: global token,code1 average, code2 average
        concat = torch.cat(
            [
                code1_sentence_h,
                code2_sentence_h,
                cls_output,
            ],
            dim=-1,
        )

        # yield logit from label classifier
        logits = self.label_classifier(concat)
        prob = nn.functional.softmax(logits)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # print(logits)
            # print(labels)
            labels = labels.squeeze(-1)
            loss = loss_fct(logits, labels)
            return loss, prob
        else:
            return prob

    def entity_average(self, hidden_output, e_mask):
        """
        Average the entity hidden state vectors (H_i ~ H_j)
        :param hidden_output: [batch_size, j-i+1, dim]
        :param e_mask: [batch_size, max_seq_len]
                e.g. e_mask[0] == [0, 0, 0, 1, 1, 1, 0, 0, ... 0]
        :return: [batch_size, dim]
        """
        e_mask_unsqueeze = e_mask.unsqueeze(1)  # [b, 1, j-i+1]
        length_tensor = (e_mask != 0).sum(dim=1).unsqueeze(1)  # [batch_size, 1]

        # [b, 1, j-i+1] * [b, j-i+1, dim] = [b, 1, dim] -> [b, dim]
        sum_vector = torch.bmm(e_mask_unsqueeze.float(), hidden_output).squeeze(1)
        avg_vector = sum_vector.float() / length_tensor.float()  # broadcasting
        return avg_vector


## CodeT5 Encoder

### "An Improved Baseline for Sentence-level Relation Extraction"(https://arxiv.org/abs/2102.01373)
### "EncT5: Fine-tuning T5 Encoder for Non-autoregressive Tasks"(https://arxiv.org/pdf/2110.08426.pdf)

마찬가지로 CodeT5도 경우는 토크나이저가 비교적 효율적이라서, 데이터셋의 82%가 512 토큰 안에 들어갑니다. PLBART와 유사하게 Encoder만 따와서 훈련을 시켰습니다.

CodeT5를 학습시킬 때 CLS 토큰뿐만 아니라 SEP 토큰의 정보까지 활용해서, 두 문장간의 관계를 추출하도록 했습니다. 논문에서는 추가적인 Punctuation을 제공하지만, 이번 대회는 코드 길이가 중요했기 때문에 CLS, SEP만 추출하는 걸로 살짝 변형해서 사용했습니다. 또한 원래 Decoder에게 정보를 넘겨주는 Layer에서 분류 작업을 하는 것이 불안정할 것을 우려하여 Sentence Average도 추출하였습니다.


In [ ]:
import copy
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.models.t5.modeling_t5 import T5EncoderModel, T5Config, T5PreTrainedModel, T5Stack
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.utils.model_parallel_utils import assert_device_map, get_device_map
from utils.heads import (
    AdaptivePooler,
    MeanPooler,
    FCLayer,
)


class VHT5EncoderForSequenceClassification(T5PreTrainedModel):
    """ 
    Using [CLS token hidden states, Input Sequence Weighted Average, SEP token hidden states] for classification
    """

    _keys_to_ignore_on_load_missing = [
        r"encoder\.embed_tokens\.weight",
    ]

    def __init__(self, config: T5Config, dropout=0.1, pooler="adaptive"):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.shared = nn.Embedding(config.vocab_size, config.d_model)

        encoder_config = copy.deepcopy(config)
        encoder_config.use_cache = False
        encoder_config.is_encoder_decoder = False
        self.encoder = T5Stack(encoder_config, self.shared)
        pooler_class = AdaptivePooler if pooler == "adaptive" else MeanPooler
        self.pooler = pooler_class(input_size=config.hidden_size)

        # FC Layers
        self.cls_fc_layer = FCLayer(
            self.config.hidden_size * 4,
            self.config.hidden_size,
            self.config.dropout_rate,
            use_activation=True,
        )

        self.sep_fc_layer = FCLayer(
            self.config.hidden_size * 4,
            self.config.hidden_size,
            self.config.dropout_rate,
            use_activation=True,
        )

        self.label_classifier = FCLayer(
            self.config.hidden_size * 3,
            self.config.num_labels,
            self.config.dropout_rate,
            use_activation=False,
        )

        # Initialize weights and apply final processing
        self.post_init()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

    def parallelize(self, device_map=None):
        self.device_map = (
            get_device_map(len(self.encoder.block), range(torch.cuda.device_count()))
            if device_map is None
            else device_map
        )
        assert_device_map(self.device_map, len(self.encoder.block))
        self.encoder.parallelize(self.device_map)
        self.classifier = self.classifier.to(self.encoder.first_device)
        self.model_parallel = True

    def deparallelize(self):
        self.encoder.deparallelize()
        self.encoder = self.encoder.to("cpu")
        self.model_parallel = False
        self.device_map = None
        torch.cuda.empty_cache()

    def get_input_embeddings(self):
        return self.shared

    def set_input_embeddings(self, new_embeddings):
        self.shared = new_embeddings
        self.encoder.set_input_embeddings(new_embeddings)

    def get_encoder(self):
        return self.encoder

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        last_token_index=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=True,
            return_dict=return_dict,
        )

        # get adaptive pooler for the given sequence
        weights, pooled_output = self.pooler(outputs[0], mask=attention_mask)

        # Global token's 4 hidden states concatenation and  projection result
        idx_seq = torch.arange(input_ids.size(0)).to(input_ids.device)

        cls_concat = torch.cat(
            tuple([outputs["hidden_states"][i][idx_seq, 0, :] for i in [-4, -3, -2, -1]]), dim=-1
        )
        cls_output = self.cls_fc_layer(cls_concat)

        sep_concat = torch.cat(
            tuple(
                [outputs["hidden_states"][i][idx_seq, last_token_index] for i in [-4, -3, -2, -1]]
            ),
            dim=-1,
        )
        sep_output = self.sep_fc_layer(sep_concat)

        concat = torch.cat([cls_output, pooled_output, sep_output], dim=-1)

        logits = self.label_classifier(concat)

        loss = None
        outputs.hidden_states = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (
                    labels.dtype == torch.long or labels.dtype == torch.int
                ):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


### Preprocessing

💬 전처리 💬
전처리 부분에서 가장 주요했던 것은 import 부분을 아예 제거하는 것이었습니다. Import는 대부분 사용을 하지 않고, 사용한다고 해도 어차피 코드에서 드러나니, import 부분을 전부 제외하자는 상하님의 아이디어는 기발했습니다 👍 

그 외에 여러 개의 space, tab 들을 한 개의 space로 제거하는 전처리를 거쳤습니다.


In [ ]:
import re


class FunctionPreprocessor:
    def __init__(self,):
        pass

    def get_function(self, code):
        results = []
        fn_list = re.findall("\ndef [a-zA-Z0-9_]+\(", code)

        for fn in fn_list:
            results.append(fn[4:-1].strip())
        return results

    def determine_function(self, code, function_name):
        num = len(re.findall("[^a-zA-Z]" + function_name + "[^a-zA-Z]", code))
        return False if num <= 1 else True

    def delete_function(self, code, name):
        start_id, _ = re.search("def " + name, code).span()
        ptr = start_id

        while ptr < len(code) - 1:
            if code[ptr] == "\n" and re.search("[a-zA-Z]", code[ptr + 1]) is not None:
                break
            ptr += 1

        if ptr != len(code) - 1:
            end_id = ptr
            code = code[:start_id] + code[end_id:]

        return code

    def preprocess(self, code):
        code = "\n" + code
        fn_list = self.get_function(code)
        if len(fn_list) == 0:
            return code

        for fn in fn_list:
            flag = self.determine_function(code, fn)

            if flag == False:
                code = self.delete_function(code, fn)

        return code

    def __call__(self, datasets):
        code1_list = []
        code2_list = []

        size = len(datasets["code1"])
        for i in range(size):
            code1 = self.preprocess(datasets["code1"][i])
            code2 = self.preprocess(datasets["code2"][i])

            code1_list.append(code1)
            code2_list.append(code2)

        datasets["code1"] = code1_list
        datasets["code2"] = code2_list
        return datasets


class AnnotationPreprocessor:
    def __init__(self,):
        pass

    def search(self, sen_list, string):
        for i, sen in enumerate(sen_list):
            if string in sen:
                return i
        return -1

    def delete_annotation_block(self, code, string):
        sens = [sen for sen in code.split("\n")]

        start_id = self.search(sens, string)
        end_id = self.search(sens[start_id + 1 :], string)
        if end_id != -1:
            end_id += start_id + 1
            code = sens[:start_id] + sens[end_id + 1 :]
        else:
            code = sens[:start_id] + sens[start_id + 1 :]

        code = "\n".join(code)
        return code

    def delete_block(self, code, string):
        while string in code:
            code = self.delete_annotation_block(code, string)
        return code

    def delete_annotation(self, code):
        sens = code.split("\n")

        sens_processed = []
        for sen in sens:
            if "#" in sen:
                index = sen.index("#")
                sen = sen[:index]
            sens_processed.append(sen)

        return "\n".join(sens_processed)

    def delete_import(self, code):
        sens = code.split("\n")

        sens_processed = []
        for sen in sens:
            if "import" not in sen:
                sens_processed.append(sen)

        return "\n".join(sens_processed)

    def preprocess(self, code):
        code = self.delete_block(code, '"""')
        code = self.delete_block(code, "'''")
        code = self.delete_annotation(code)
        code = self.delete_import(code)
        code = re.sub("\s+", " ", code).strip()
        return code

    def __call__(self, datasets):
        code1_list = []
        code2_list = []

        size = len(datasets["code1"])
        for i in range(size):
            code1 = self.preprocess(datasets["code1"][i])
            code2 = self.preprocess(datasets["code2"][i])

            code1_list.append(code1)
            code2_list.append(code2)

        datasets["code1"] = code1_list
        datasets["code2"] = code2_list
        return datasets


class BasePreprocessor:
    def __init__(self,):
        pass

    def search(self, sen_list, string):
        for i, sen in enumerate(sen_list):
            if string in sen:
                return i
        return -1

    def delete_annotation_block(self, code, string):
        sens = [sen for sen in code.split("\n")]

        start_id = self.search(sens, string)
        end_id = self.search(sens[start_id + 1 :], string)
        if end_id != -1:
            end_id += start_id + 1
            code = sens[:start_id] + sens[end_id + 1 :]
        else:
            code = sens[:start_id] + sens[start_id + 1 :]

        code = "\n".join(code)
        return code

    def delete_block(self, code, string):
        while string in code:
            code = self.delete_annotation_block(code, string)
        return code

    def delete_annotation(self, code):
        sens = code.split("\n")

        sens_processed = []
        for sen in sens:
            if "#" in sen:
                index = sen.index("#")
                sen = sen[:index]
            sens_processed.append(sen)

        return "\n".join(sens_processed)

    def delete_import(self, code):
        sens = code.split("\n")

        sens_processed = []
        for sen in sens:
            if "import" not in sen:
                sens_processed.append(sen)

        return "\n".join(sens_processed)

    def preprocess(self, code):
        code = self.delete_block(code, '"""')
        code = self.delete_block(code, "'''")
        code = self.delete_annotation(code)
        code = self.delete_import(code)
        # code = re.sub('\s+', ' ', code).strip()
        return code

    def __call__(self, datasets):
        code1_list = []
        code2_list = []
        label_list = []

        size = len(datasets["code1"])
        for i in range(size):
            code1 = self.preprocess(datasets["code1"][i])
            code2 = self.preprocess(datasets["code2"][i])
            if "similar" in datasets:
                label_list = [int(x) for x in datasets["similar"]]

            code1_list.append(code1)
            code2_list.append(code2)

        datasets["code1"] = code1_list
        datasets["code2"] = code2_list
        if "similar" in datasets:
            datasets["similar"] = label_list

        return datasets

### 데이터 제작 코드

In [ ]:
import os
import shutil
import glob
from itertools import combinations
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from datasets import Dataset, DatasetDict


# read .py documents
df_train = pd.DataFrame({}, columns=["file_name", "question_label", "code"])
TRAIN_PATH = "/home/ubuntu/plclassification/code"
list_questions = glob.glob(os.path.join(TRAIN_PATH, "*"))

list_index = []
list_problem_names = []
list_code_text = []

for index, question in enumerate(list_questions):
    problem_name = os.path.basename(question)
    # extract numerics only from problem_name
    problem_name = "".join(i for i in problem_name if i.isdigit())

    for file in glob.glob(os.path.join(question, "*.py")):
        file_base_name = os.path.basename(file)
        # read text from .py file
        with open(file, "r") as f:
            code_text = f.read()
            # append row with index, problem_name and code text
            list_index.append(file_base_name)
            list_problem_names.append(problem_name)
            list_code_text.append(code_text)

df_train["file_name"] = list_index
df_train["question_label"] = list_problem_names
# remove 0 in front of the number in question_label
df_train["question_label"] = df_train["question_label"].str.replace("^0+", "").astype(int)
df_train["code"] = list_code_text

# create similar pairs
df_similar = pd.DataFrame(
    {}, columns=["code1", "code2", "similar", "code1_group", "code2_group", "pair_id"]
)

for i in tqdm(range(1, 301)):
    list_code1 = []
    list_code2 = []
    list_question_pair_id = []
    list_group_id = []
    # get corresponding dataframe where question_label == i
    group_index = df_train[df_train["question_label"] == i].index
    df_group = df_train.iloc[group_index]

    # make unique pair combination from group_index list
    combinated_indices = list(combinations(group_index, 2))
    for index in combinated_indices:
        code1 = df_train.iloc[index[0]]["code"]
        code2 = df_train.iloc[index[1]]["code"]
        list_code1.append(code1)
        list_code2.append(code2)
        question_pair_id = index[0] ** 3 + index[1] ** 3 + index[0] + index[1]
        list_question_pair_id.append(question_pair_id)

    df_similar = df_similar.append(
        pd.DataFrame(
            {
                "code1": list_code1,
                "code2": list_code2,
                "similar": 1,
                "code1_group": i,
                "code2_group": i,
                "pair_id": list_question_pair_id,
            }
        )
    )

df_similar = df_similar.reset_index(drop=True)
df_similar["code1_group"] = df_similar["code1_group"].astype(int)
df_similar["code2_group"] = df_similar["code2_group"].astype(int)
print(df_similar.shape)

# Create different pairs
NUM = 120000
df_different = pd.DataFrame({}, columns=["code1", "code2", "similar"])

for i in tqdm(range(1, 301)):
    # get corresponding dataframe where question_label == i
    group_index = df_train[df_train["question_label"] == i].index
    other_index = df_train[df_train["question_label"] != i].index
    sample_index = np.random.choice(group_index, NUM, replace=True)
    other_sample_index = np.random.choice(other_index, NUM, replace=True)

    # make unique pair id: multiplication - sum
    question_pair_id = (
        sample_index ** 3 + other_sample_index ** 3 + (sample_index + other_sample_index)
    )
    # make group pair id
    other_group_index = df_train.iloc[other_sample_index]["question_label"].values

    # append df_train.iloc[sample_index]["code"] to df_different["code1"]
    code1 = df_train.iloc[sample_index]["code"].values
    code2 = df_train.iloc[other_sample_index]["code"].values
    df_different = df_different.append(
        pd.DataFrame(
            {
                "code1": code1,
                "code2": code2,
                "similar": 0,
                "code1_group": i,
                "code2_group": other_group_index.astype(int),
                "question_pair_id": question_pair_id,
            }
        )
    )

df_different = df_different.drop_duplicates(subset=["question_pair_id"])
df_different["code1_group"] = df_different["code1_group"].astype(int)
df_different["code2_group"] = df_different["code2_group"].astype(int)
print(df_different.shape)


# import KFold from sklearn
list_groups = list([i for i in range(1, 301)])
kf = KFold(n_splits=5, shuffle=True)
folds_10 = kf.get_n_splits(list_groups)

for fold_index, (train_index, test_index) in enumerate(kf.split(list_groups)):

    df_similar_train_fold = df_similar[df_similar["code1_group"].isin(train_index)]
    df_similar_val_fold = df_similar[df_similar["code1_group"].isin(test_index)]

    df_different_train_fold = df_different[
        df_different["code1_group"].isin(train_index)
        & df_different["code2_group"].isin(train_index)
    ]
    df_different_train_fold = df_different_train_fold.sample(len(df_similar_train_fold))
    df_different_val_fold = df_different[
        df_different["code1_group"].isin(test_index) & df_different["code2_group"].isin(test_index)
    ]
    df_different_val_fold = df_different_val_fold.sample(len(df_similar_val_fold))

    df_train = pd.concat([df_similar_train_fold, df_different_train_fold])
    # shuffle df_train
    df_train = df_train.sample(frac=1)
    # reset index for df_train
    df_train = df_train.reset_index(drop=True)

    df_val = pd.concat([df_similar_val_fold, df_different_val_fold])
    # shuffle df_val
    df_val = df_val.sample(frac=1)
    # reset index for df_val
    df_val = df_val.reset_index(drop=True)

    print(df_train.shape, df_val.shape)
    assert df_train.columns == df_val.columns

    dataset_fold = DatasetDict()
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_fold["train"] = dataset_train
    dataset_fold["val"] = dataset_val

    name = f"PoolC/{fold_index+1}-fold-clone-detection-600k-5fold"
    dataset_fold.push_to_hub(name, private=True)

미리 업로드한 데이터셋에서 90만개를 랜덤추출을 했기 때문에, Fold 별로 갈라진 Class는 동일하지만 추출된 90만 개의 데이터셋은 각 모델 별로 상이합니다. 이를 통해서 앙상블 성능을 높이려고 했습니다.

> 작성자: 박상하, 안영진

> This Notebook has been released under the Apache 2.0 open source license.

댓글로 의견을 공유해주시면 저희에게도 공부할 기회가 될 것 같습니다 🙏
감사합니다!